In [79]:
import pandas as pd
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon, init
from mxnet.gluon import nn, rnn
import gluonnlp as nlp
import pkuseg
import multiprocessing as mp
import time
from d2l import try_gpu
import itertools
from sklearn.metrics import accuracy_score, f1_score
# fixed random number seed
np.random.seed(9102)
mx.random.seed(9102)

In [80]:
DATA_FOLDER = 'data/'
TRAIN_DATA = 'train.csv'
WORD_EMBED = 'sgns.weibo.bigram-char'
LABEL_FILE = 'train.label'
N_ROWS=10000
ctx = try_gpu()
seg = pkuseg.pkuseg(model_name='web')

In [81]:
train_df = pd.read_csv(DATA_FOLDER+TRAIN_DATA, sep='|')

In [82]:
dataset =[ [row[0], row[1]] for _, row in train_df.iterrows()]
train_dataset, valid_dataset = nlp.data.train_valid_split(dataset, .1)
len(train_dataset), len(valid_dataset)

(776847, 86317)

In [83]:
def tokenizer(x):
    tweet, label = x
    if type(tweet) != str:
        tweet = str(tweet)
    word_list = seg.cut(tweet)
    return word_list, label

def get_length(x):
    return float(len(x[0]))

def to_word_list(dataset):
    start = time.time()
    with mp.Pool() as pool:
        # Each sample is processed in an asynchronous manner.
        dataset = gluon.data.ArrayDataset(pool.map(tokenizer, dataset))
        lengths = gluon.data.ArrayDataset(pool.map(get_length, dataset))
    end = time.time()

    print('Done! Tokenizing Time={:.2f}s, #Sentences={}'.format(end - start, len(dataset)))
    return dataset, lengths

train_word_list, train_word_lengths = to_word_list(train_dataset)
valid_word_list, valid_word_lengths = to_word_list(valid_dataset)

Done! Tokenizing Time=86.11s, #Sentences=776847
Done! Tokenizing Time=10.47s, #Sentences=86317


In [84]:
train_seqs = [sample[0] for sample in train_word_list]
counter = nlp.data.count_tokens(list(itertools.chain.from_iterable(train_seqs)))

vocab = nlp.Vocab(counter, max_size=100000)

# load customed pre-trained embedding
embedding_weights = nlp.embedding.TokenEmbedding.from_file(file_path=DATA_FOLDER+WORD_EMBED)
vocab.set_embedding(embedding_weights)
print(vocab)

def token_to_idx(x):
    return vocab[x[0]], x[1]

# A token index or a list of token indices is returned according to the vocabulary.
with mp.Pool() as pool:
    train_dataset = pool.map(token_to_idx, train_word_list)
    valid_dataset = pool.map(token_to_idx, valid_word_list)

/home/steven/miniconda3/envs/dl/lib/python3.7/site-packages/gluonnlp/embedding/token_embedding.py:296: UserWarning: line 0 in data/sgns.weibo.bigram-char: skipped likely header line.
  .format(line_num, pretrained_file_path))


Vocab(size=100004, unk="<unk>", reserved="['<pad>', '<bos>', '<eos>']")


In [85]:
batch_size = 64
bucket_num = 10
bucket_ratio = 0.5


def get_dataloader():
    # Construct the DataLoader Pad data, stack label and lengths
    batchify_fn = nlp.data.batchify.Tuple(nlp.data.batchify.Pad(axis=0), \
                                          nlp.data.batchify.Stack())

    # in this example, we use a FixedBucketSampler,
    # which assigns each data sample to a fixed bucket based on its length.
    batch_sampler = nlp.data.sampler.FixedBucketSampler(
        train_word_lengths,
        batch_size=batch_size,
        num_buckets=bucket_num,
        ratio=bucket_ratio,
        shuffle=True)
    print(batch_sampler.stats())

    # train_dataloader
    train_dataloader = gluon.data.DataLoader(
        dataset=train_dataset,
        batch_sampler=batch_sampler,
        batchify_fn=batchify_fn)
    # valid_dataloader
    valid_dataloader = gluon.data.DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        batchify_fn=batchify_fn)
    return train_dataloader, valid_dataloader

train_dataloader, valid_dataloader = get_dataloader()

FixedBucketSampler:
  sample_num=776847, batch_num=4617
  key=[22, 35, 48, 61, 74, 87, 100, 113, 126, 139]
  cnt=[637662, 69980, 31150, 17042, 11028, 7082, 2748, 149, 3, 3]
  batch_size=[202, 127, 92, 72, 64, 64, 64, 64, 64, 64]


In [86]:
for tweet, label in train_dataloader:
    print(tweet, label)
    break


[[0.0000e+00 0.0000e+00 0.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [4.9000e+02 5.0000e+00 1.4615e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.5000e+01 9.2900e+02 5.0000e+00 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 ...
 [6.3300e+02 5.1000e+01 3.2100e+02 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.2600e+02 1.6300e+03 1.1996e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [1.0500e+02 4.2000e+01 1.4000e+01 ... 0.0000e+00 0.0000e+00 0.0000e+00]]
<NDArray 202x22 @cpu_shared(0)> 
[34 18 10 15  3 29 31 30 17  3 14  1 34 42 27 28 27 34 14 29 34 34 23 14
 31 12 34 35 13 29 31 12 40 19 42 29 16  3 50 34 70  6 19 14 27 35 17 12
 35 60 15 41 39 12 61 58 70 22 31 23  9 23 24  6 14 20 12  1 29 40 15 43
 55 15 31  3  1  3 29 17  3 37 14 16 29 13 12 47 30  3  3 62 36 33 35 10
 52 29 12  1 14 33  3  3  3 12 16 15 14 16 29 15 31 60  3 31 59  3 19 34
 53 27 35 23 19 35 40 35 25 52 34 27 70 60 27  9 70 52 10 35 18 34 12 22
 31 34 50  3 16  3 17 14 32 34 31 52 34  3 29 38  3  3  3 28 24 70 14 14
 37 29 35 1

## Model contruction
Self attention layer, weighted cross entropy, and whole model

In [87]:
# custom attention layer
# in this class, we want to implement the operation:
# softmax(W_2 * tanh(W_1 * H))
# where H is the word embedding of the whole sentence, of shape (num_of_word, embed_size)
class SelfAttention(nn.HybridBlock):
    def __init__(self, att_unit, att_hops, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)
        with self.name_scope():
            # this layer is tanh(w_1 * H), the att_unit corresponds to d_a in the essay
            self.ut_dense = nn.Dense(att_unit, activation='relu', flatten=False)
            # this layer implements the multiple hops
            self.et_dense = nn.Dense(att_hops, activation=None, flatten=False)

    def hybrid_forward(self, F, x): # F is the backend which implements the tensor operation
        # x shape: [batch_size, seq_len, embedding_width]
        # ut shape: [batch_size, seq_len, att_unit]
        ut = self.ut_dense(x) # batch_size * seq_len [* embed_size * embed_size *] att_unit
        # et shape: [batch_size, seq_len, att_hops]
        et = self.et_dense(ut)# batch_size * seq_len [* att_unit * att_unit *] att_hops

        # att shape: [batch_size,  att_hops, seq_len]
        # softmax is performed along the seq_len dimension
        att = F.softmax(F.transpose(et, axes=(0, 2, 1)), axis=-1)
        # output shape [batch_size, att_hops, embedding_width]
        output = F.batch_dot(att, x)
        # output is the weighted matrix representation of the matrix
        # att is the weighted vector we use as attention
        return output, att
    
# d_a = 20, hops = 5
print(SelfAttention(20, 5))

SelfAttention(
  (ut_dense): Dense(None -> 20, Activation(relu))
  (et_dense): Dense(None -> 5, linear)
)


In [88]:
class WeightedSoftmaxCE(nn.HybridBlock):
    def __init__(self, sparse_label=True, from_logits=False,  **kwargs):
        super(WeightedSoftmaxCE, self).__init__(**kwargs)
        with self.name_scope():
            self.sparse_label = sparse_label
            self.from_logits = from_logits

    def hybrid_forward(self, F, pred, label, class_weight, depth=None):
        if self.sparse_label:
            label = F.reshape(label, shape=(-1, ))
            label = F.one_hot(label, depth)
        if not self.from_logits:
            pred = F.log_softmax(pred, -1)

        weight_label = F.broadcast_mul(label, class_weight)
        loss = -F.sum(pred * weight_label, axis=-1)

        # return F.mean(loss, axis=0, exclude=True)
        return loss

In [89]:
class SelfAttentiveBiLSTM(nn.HybridBlock):
    def __init__(self, vocab_len, embsize, nhidden, nlayers, natt_unit, natt_hops, \
                 nfc, nclass, # these two params are not used currrently
                 drop_prob, pool_way, prune_p=None, prune_q=None, **kwargs):
        super(SelfAttentiveBiLSTM, self).__init__(**kwargs)
        with self.name_scope():
            # now we switch back to shared layers
            self.embedding_layer = nn.Embedding(vocab_len, embsize)
            
            self.bilstm = rnn.GRU(nhidden, num_layers=nlayers, dropout=drop_prob, \
                                        bidirectional=True)
            
            self.att_encoder = SelfAttention(natt_unit, natt_hops)
            self.dense = nn.Dense(nfc, activation='relu')
            # this layer is used to output the final class
            self.output_layer = nn.Dense(nclass)
            
            self.dense_p, self.dense_q = None, None
            if all([prune_p, prune_q]):
                self.dense_p = nn.Dense(prune_p, activation='relu', flatten=False)
                self.dense_q = nn.Dense(prune_q, activation='relu', flatten=False)

            self.drop_prob = drop_prob
            self.pool_way = pool_way

    def hybrid_forward(self, F, inp):
        # inp_embed size: [batch, seq_len, embed_size]
        inp_embed = self.embedding_layer(inp)
        # rnn requires the first dimension to be the time steps
        h_output = self.bilstm(F.transpose(inp_embed, axes=(1, 0, 2)))
        # att_output: [batch, att_hops, emsize]
        output, att = self.att_encoder(F.transpose(h_output, axes=(1, 0, 2)))
        '''
        FIXME: now this code will only work with flatten
        '''
        dense_input = None
        if self.pool_way == 'flatten':
            dense_input = F.Dropout(F.flatten(output), self.drop_prob)
        else:
            raise NotImplemented
        '''
        elif self.pool_way == 'mean':
            dense_input = F.Dropout(F.mean(att_output, axis=1), self.drop_prob)
        elif self.pool_way == 'prune' and all([self.dense_p, self.dense_q]):
            # p_section: [batch, att_hops, prune_p]
            p_section = self.dense_p(att_output)
            # q_section: [batch, emsize, prune_q]
            q_section = self.dense_q(F.transpose(att_output, axes=(0, 2, 1)))
            dense_input = F.Dropout(F.concat(F.flatten(p_section), F.flatten(q_section), dim=-1), self.drop_prob)
        '''
        dense_out = self.dense(dense_input)
        output = self.output_layer(F.Dropout(dense_out, self.drop_prob))

        return output, att

In [90]:
vocab_len = len(vocab)
emsize = 300   # word embedding size
nhidden = 400    # lstm hidden_dim
nlayers = 4     # lstm layers
natt_unit = 400     # the hidden_units of attention layer
natt_hops = 20    # the channels of attention
nfc = 512  # last dense layer size
nclass = 72 # we have 72 emoji in total

drop_prob = 0.3
pool_way = 'flatten'    # # The way to handle M
prune_p = None
prune_q = None

ctx = try_gpu()

try:
    assert(False)
    model = gluon.nn.SymbolBlock.imports("model/model-symbol.json", ['data'], \
                                         "model/model-0001.params", ctx=ctx)
    print('use saved model params to start')
except:
    model = SelfAttentiveBiLSTM(vocab_len, emsize, nhidden, nlayers,
                            natt_unit, natt_hops, nfc, nclass,
                            drop_prob, pool_way, prune_p, prune_q)

    print('initialize a new model')
    model.initialize(init=init.Xavier(), ctx=ctx)
    model.hybridize()

    # Attach a pre-trained glove word vector to the embedding layer
    model.embedding_layer.weight.set_data(vocab.embedding.idx_to_vec)
    # fixed the embedding layer
    model.embedding_layer.collect_params().setattr('grad_req', 'null')

print(model)

initialize a new model
SelfAttentiveBiLSTM(
  (embedding_layer): Embedding(100004 -> 300, float32)
  (bilstm): GRU(None -> 400, TNC, num_layers=4, dropout=0.3, bidirectional)
  (att_encoder): SelfAttention(
    (ut_dense): Dense(None -> 400, Activation(relu))
    (et_dense): Dense(None -> 20, linear)
  )
  (dense): Dense(None -> 512, Activation(relu))
  (output_layer): Dense(None -> 72, linear)
)


In [91]:
vocab.embedding.idx_to_vec[vocab.embedding.token_to_idx['i']]


[-0.216152  0.111755  0.131167  0.108303 -0.56618   0.154915 -0.682917
  0.184372 -1.259019 -0.270754 -0.295431 -0.477648  0.313637 -0.249563
  0.464083 -0.177201  0.250866  0.63421  -0.023141 -0.083413 -0.086886
  0.373867 -0.120577  0.154108  0.075599  0.749676  0.064633  0.346573
 -0.375857  0.170967 -0.387877  0.621435  0.252638 -0.327384  0.03449
  0.59719   0.396766  0.550666 -0.658407 -0.525238  0.167532 -0.511287
 -0.360124 -0.815612 -0.511149 -0.866398  0.068793 -0.629899  0.036555
 -0.245903 -0.501821  0.222177 -0.887551 -0.059061  0.357666  0.444045
 -0.632446  0.706885  0.488229  0.459782  0.109316 -0.090775 -0.408769
  0.25539   0.630114 -0.136657 -0.541437  0.510262 -0.273591  0.137092
 -0.586211 -0.199848  0.066356  0.603941 -0.288794 -0.023497 -0.258354
  0.341849  0.427584 -0.001543  0.755926  0.718712 -1.017008  0.452808
  0.173271  0.29188   0.644698  0.49299   0.216398  0.517657  0.00933
  0.858918 -0.384057 -0.178975 -0.281533  1.395328  0.856061  0.560499
 -0.335

## Training helpers
Calculate loss, one epoch computation and top function for train and valid

In [92]:
def calculate_loss(x, y, model, loss, class_weight, penal_coeff):
    pred, att = model(x)
    y = nd.array(y.asnumpy().astype('int32')).as_in_context(ctx)
    if loss_name in ['sce', 'l1', 'l2']:
        l = loss(pred, y)
    elif loss_name == 'wsce':
        l = loss(pred, y, class_weight, class_weight.shape[0])
    else:
        raise NotImplemented
    # penalty, now we have two att's
    diversity_penalty = nd.batch_dot(att, nd.transpose(att, axes=(0, 2, 1))) - \
                        nd.eye(att.shape[1], ctx=att.context)
    l = l + penal_coeff * diversity_penalty.norm(axis=(1, 2))

    return pred, l

In [93]:
def one_epoch(data_iter, model, loss, trainer, ctx, is_train, epoch,
              penal_coeff=0.0, clip=None, class_weight=None, loss_name='sce'):

    loss_val = 0.
    total_pred = []
    total_true = []
    n_batch = 0

    for batch_x, batch_y in data_iter:
        batch_x = batch_x.as_in_context(ctx)
        batch_y = batch_y.as_in_context(ctx)

        if is_train:
            with autograd.record():
                batch_pred, l = calculate_loss(batch_x, batch_y, model, \
                                               loss, class_weight, penal_coeff)

            # backward calculate
            l.backward()

            # clip gradient
            clip_params = [p.data() for p in model.collect_params().values()]
            if clip is not None:
                norm = nd.array([0.0], ctx)
                for param in clip_params:
                    if param.grad is not None:
                        norm += (param.grad ** 2).sum()
                norm = norm.sqrt().asscalar()
                if norm > clip:
                    for param in clip_params:
                        if param.grad is not None:
                            param.grad[:] *= clip / norm

            # update parmas
            trainer.step(batch_x.shape[0])

        else:
            batch_pred, l = calculate_loss(batch_x, batch_y, model, \
                                           loss, class_weight, penal_coeff)

        # keep result for metric
        batch_pred = nd.argmax(nd.softmax(batch_pred, axis=1), axis=1).asnumpy()
        batch_true = np.reshape(batch_y.asnumpy(), (-1, ))
        total_pred.extend(batch_pred.tolist())
        total_true.extend(batch_true.tolist())
        
        batch_loss = l.mean().asscalar()

        n_batch += 1
        loss_val += batch_loss

        # check the result of traing phase
        if is_train and n_batch % 400 == 0:
            print('epoch %d, batch %d, batch_train_loss %.4f, batch_train_acc %.3f' %
                  (epoch, n_batch, batch_loss, accuracy_score(batch_true, batch_pred)))

    # metric
    F1 = f1_score(np.array(total_true), np.array(total_pred), average='weighted')
    acc = accuracy_score(np.array(total_true), np.array(total_pred))
    loss_val /= n_batch

    if is_train:
        print('epoch %d, learning_rate %.5f \n\t train_loss %.4f, acc_train %.3f, F1_train %.3f, ' %
              (epoch, trainer.learning_rate, loss_val, acc, F1))
        # declay lr
        if epoch % 3 == 0:
            trainer.set_learning_rate(trainer.learning_rate * 0.9)
    else:
        print('\t valid_loss %.4f, acc_valid %.3f, F1_valid %.3f, ' % (loss_val, acc, F1))

In [94]:
def train_valid(data_iter_train, data_iter_valid, model, loss, trainer, ctx, nepochs,
                penal_coeff=0.0, clip=None, class_weight=None, loss_name='sce'):

    for epoch in range(1, nepochs+1):
        start = time.time()
        # train
        is_train = True
        one_epoch(data_iter_train, model, loss, trainer, ctx, is_train,
                  epoch, penal_coeff, clip, class_weight, loss_name)

        # valid
        is_train = False
        one_epoch(data_iter_valid, model, loss, trainer, ctx, is_train,
                  epoch, penal_coeff, clip, class_weight, loss_name)
        end = time.time()
        print('time %.2f sec' % (end-start))
        print("*"*100)

## Train
Now we will train this model. To handle data inbalance, we first set an estimated weight of the labels.

In [101]:
from util import get_weight
weight_list = get_weight(DATA_FOLDER, LABEL_FILE)
print(weight_list)
class_weight = None
loss_name = 'sce'
optim = 'adam'
lr = 0.001
penal_coeff = 0.0003
clip = .5
nepochs = 5

trainer = gluon.Trainer(model.collect_params(), optim, {'learning_rate': lr})

if loss_name == 'sce':
    loss = gluon.loss.SoftmaxCrossEntropyLoss()
elif loss_name == 'wsce':
    loss = WeightedSoftmaxCE()
    # the value of class_weight is obtained by counting data in advance. It can be seen as a hyperparameter.
    class_weight = nd.array(weight_list, ctx=ctx)
elif loss_name == 'l1':
    loss = gluon.loss.L1Loss()
elif loss_name == 'l2':
    loss = gluon.loss.L2Loss()

[9.71540487119179, 6.231636866323191, 8.231834498039241, 1.0, 18.57280561073304, 17.401745026992835, 13.552092608433227, 18.37972211720023, 15.594356705916198, 14.53901176939084, 13.104536718725747, 16.52330626689046, 10.602513556421325, 15.574148561247236, 11.712361031570136, 11.957176969460722, 14.323192522444323, 13.140910385186368, 13.627624950204199, 9.743139048861558, 16.3185802891442, 17.339836775281242, 15.665023296713656, 11.568132631262811, 13.860529807325692, 17.879875825858495, 12.824470864344555, 9.695774830353102, 12.334632996030269, 10.868524596249232, 14.865853919439653, 13.99389658605509, 9.691551652607913, 14.552353595700842, 15.507316853017446, 15.917054133678395, 15.466452578238123, 14.226260120483808, 14.093314603527972, 13.490458202189915, 9.049913740536173, 16.70756081057398, 15.640214579530252, 15.937696994349995, 15.28061912035655, 16.913653306770467, 19.06601879298311, 15.01772544677559, 14.223976363593618, 20.370409760379705, 12.195038400645434, 19.1006179270

In [102]:
# train and valid
train_valid(train_dataloader, valid_dataloader, model, loss, \
            trainer, ctx, nepochs, penal_coeff=penal_coeff, \
            clip=clip, class_weight=class_weight, loss_name=loss_name)
token = str(round(time.time()))
model.export("model/self-att"+token+'.params', epoch=1)

epoch 1, batch 400, batch_train_loss 3.3347, batch_train_acc 0.205
epoch 1, batch 800, batch_train_loss 3.2992, batch_train_acc 0.220
epoch 1, batch 1200, batch_train_loss 3.5468, batch_train_acc 0.134
epoch 1, batch 1600, batch_train_loss 3.4170, batch_train_acc 0.158
epoch 1, batch 2000, batch_train_loss 2.9928, batch_train_acc 0.264
epoch 1, batch 2400, batch_train_loss 3.4491, batch_train_acc 0.153
epoch 1, batch 2800, batch_train_loss 3.3380, batch_train_acc 0.163
epoch 1, batch 3200, batch_train_loss 3.3341, batch_train_acc 0.203
epoch 1, batch 3600, batch_train_loss 3.3482, batch_train_acc 0.183
epoch 1, batch 4000, batch_train_loss 3.3876, batch_train_acc 0.183
epoch 1, batch 4400, batch_train_loss 3.0698, batch_train_acc 0.181
epoch 1, learning_rate 0.00100 
	 train_loss 3.3303, acc_train 0.181, F1_train 0.119, 
	 valid_loss 3.3811, acc_valid 0.176, F1_valid 0.109, 
time 799.34 sec
************************************************************************************************

In [103]:
TEST_DATA = 'test.csv'
predictions = []
test_df = pd.read_csv(DATA_FOLDER+TEST_DATA, header=None, sep='\t')
len(test_df)

200000

In [105]:
start = time.time()
for _, tweet in test_df.iterrows():
    token = vocab[seg.cut(tweet[1])]
    if type(token) != str:
        token = [0]
    inp = nd.array(token, ctx=ctx).reshape(1,-1)
    pred, _ = model(inp)
    pred = nd.argmax(pred, axis=-1).asscalar()
    predictions.append(int(pred))
    if len(predictions)%2000==0:
        ckpt = time.time()
        print('current pred len %d, time %.2fs' % (len(predictions), ckpt-start))
        start = ckpt
submit = pd.DataFrame({'Expected': predictions})
submit.to_csv('submission.csv', sep=',', index_label='ID')

current pred len 54000, time 7.09s
current pred len 56000, time 21.73s
current pred len 58000, time 21.79s
current pred len 60000, time 21.72s
current pred len 62000, time 21.78s
current pred len 64000, time 21.74s
current pred len 66000, time 21.67s
current pred len 68000, time 21.65s
current pred len 70000, time 21.63s
current pred len 72000, time 21.66s


KeyboardInterrupt: 